# 

# Data Science Capstone Project

<p>In this project we will analyze data about car accidents and will try to predict the severity of a new collision, if it's fatal or not</p>

### Importing libraries

In [1]:
# Pandas to handle dataset
import pandas as pd 
# Numpy to handle and operate through arrays
import numpy as np
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
df_collisions = pd.read_csv("https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv")

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


 ### Data understanding

In [4]:
# Useless columns: 'OBJECTID', 'X', 'Y', 'REPORTNO', 'INCKEY', 'COLDETKEY', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'INCDATE', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC', 'SDOTCOLNUM', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY'

In [5]:
useless_columns = ['OBJECTID', 'X', 'Y', 'REPORTNO', 'STATUS', 'INCDTTM', 'INCKEY', 'COLDETKEY', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'INCDATE', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC', 'SDOTCOLNUM', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY']

df_collisions.drop(columns=useless_columns, inplace=True)

print(df_collisions.columns)

Index(['SEVERITYCODE', 'ADDRTYPE', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT',
       'PEDCYLCOUNT', 'VEHCOUNT', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER',
       'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SPEEDING', 'HITPARKEDCAR'],
      dtype='object')


In [6]:
df_collisions.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,HITPARKEDCAR
0,2,Intersection,Angles,2,0,0,2,NaN,N,Overcast,Wet,Daylight,NaN,NaN,N
1,1,Block,Sideswipe,2,0,0,2,NaN,0,Raining,Wet,Dark - Street Lights On,NaN,NaN,N
2,1,Block,Parked Car,4,0,0,3,NaN,0,Overcast,Dry,Daylight,NaN,NaN,N
3,1,Block,Other,3,0,0,3,NaN,N,Clear,Dry,Daylight,NaN,NaN,N
4,2,Intersection,Angles,2,0,0,2,NaN,0,Raining,Wet,Daylight,NaN,NaN,N


In [7]:
df_collisions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 15 columns):
SEVERITYCODE      194673 non-null int64
ADDRTYPE          192747 non-null object
COLLISIONTYPE     189769 non-null object
PERSONCOUNT       194673 non-null int64
PEDCOUNT          194673 non-null int64
PEDCYLCOUNT       194673 non-null int64
VEHCOUNT          194673 non-null int64
INATTENTIONIND    29805 non-null object
UNDERINFL         189789 non-null object
WEATHER           189592 non-null object
ROADCOND          189661 non-null object
LIGHTCOND         189503 non-null object
PEDROWNOTGRNT     4667 non-null object
SPEEDING          9333 non-null object
HITPARKEDCAR      194673 non-null object
dtypes: int64(5), object(10)
memory usage: 22.3+ MB


In [8]:
df_collisions.isnull().sum()

SEVERITYCODE           0
ADDRTYPE            1926
COLLISIONTYPE       4904
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
INATTENTIONIND    164868
UNDERINFL           4884
WEATHER             5081
ROADCOND            5012
LIGHTCOND           5170
PEDROWNOTGRNT     190006
SPEEDING          185340
HITPARKEDCAR           0
dtype: int64

In [9]:
# Total observations: 194673
# Lets drop columns that have half of this value or more of missing values (97336+)
print('Inital total columns: ' + str(np.asarray(df_collisions.columns).size))
df_collisions.dropna(axis=1, thresh=97336, inplace=True)
print('Total columns after dropping: ' + str(np.asarray(df_collisions.columns).size))

Inital total columns: 15
Total columns after dropping: 12


In [10]:
df_collisions.isnull().sum()

SEVERITYCODE        0
ADDRTYPE         1926
COLLISIONTYPE    4904
PERSONCOUNT         0
PEDCOUNT            0
PEDCYLCOUNT         0
VEHCOUNT            0
UNDERINFL        4884
WEATHER          5081
ROADCOND         5012
LIGHTCOND        5170
HITPARKEDCAR        0
dtype: int64

<p>Now lets handle object variables, and perform some data transformation so we can check for correlations</p>

<p>We need to handle those missing values</p>

In [11]:
cols_with_missing_values = ['ADDRTYPE', 'COLLISIONTYPE', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND']

df_collisions[cols_with_missing_values].head()

,ADDRTYPE,COLLISIONTYPE,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND
0,Intersection,Angles,N,Overcast,Wet,Daylight
1,Block,Sideswipe,0,Raining,Wet,Dark - Street Lights On
2,Block,Parked Car,0,Overcast,Dry,Daylight
3,Block,Other,N,Clear,Dry,Daylight
4,Intersection,Angles,0,Raining,Wet,Daylight


We can see that UNDERINFL has numeric and string values, so we need to handle it individually and carefully. Other columns has only string values.

In [12]:
df_collisions['UNDERINFL'] = df_collisions['UNDERINFL'].map({'0': 'No', '1': 'Yes', 'N': 'No', 'Y': 'Yes'})

In [13]:
df_collisions['ADDRTYPE'].fillna(df_collisions['ADDRTYPE'].mode()[0], inplace=True)
df_collisions['COLLISIONTYPE'].fillna(df_collisions['COLLISIONTYPE'].mode()[0], inplace=True)
df_collisions['UNDERINFL'].fillna(df_collisions['UNDERINFL'].mode()[0], inplace=True)
df_collisions['WEATHER'].fillna(df_collisions['WEATHER'].mode()[0], inplace=True)
df_collisions['ROADCOND'].fillna(df_collisions['ROADCOND'].mode()[0], inplace=True)
df_collisions['LIGHTCOND'].fillna(df_collisions['LIGHTCOND'].mode()[0], inplace=True)

#### Now, lets use OneHotEncoding and get_dummies to handle categorical values

In [14]:
df_collisions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 12 columns):
SEVERITYCODE     194673 non-null int64
ADDRTYPE         194673 non-null object
COLLISIONTYPE    194673 non-null object
PERSONCOUNT      194673 non-null int64
PEDCOUNT         194673 non-null int64
PEDCYLCOUNT      194673 non-null int64
VEHCOUNT         194673 non-null int64
UNDERINFL        194673 non-null object
WEATHER          194673 non-null object
ROADCOND         194673 non-null object
LIGHTCOND        194673 non-null object
HITPARKEDCAR     194673 non-null object
dtypes: int64(5), object(7)
memory usage: 17.8+ MB


#####  One-hot Encoding

In [15]:
#ADDRTYPE
df_collisions.ADDRTYPE.value_counts()

Block           128852
Intersection     65070
Alley              751
Name: ADDRTYPE, dtype: int64

In [16]:
#COLLISIONTYPE
df_collisions.COLLISIONTYPE.value_counts()

Parked Car    52891
Angles        34674
Rear Ended    34090
Other         23703
Sideswipe     18609
Left Turn     13703
Pedestrian     6608
Cycles         5415
Right Turn     2956
Head On        2024
Name: COLLISIONTYPE, dtype: int64

In [17]:
#UNDERINFL
df_collisions.UNDERINFL.value_counts()

No     185552
Yes      9121
Name: UNDERINFL, dtype: int64

In [18]:
#COLLISIONTYPE
df_collisions.HITPARKEDCAR.value_counts()

N    187457
Y      7216
Name: HITPARKEDCAR, dtype: int64

Since there's no number sequencing, we can use get_dummies in the columns above

In [19]:
object_columns = ['ADDRTYPE', 'COLLISIONTYPE', 'UNDERINFL', 'HITPARKEDCAR']
df_collisions = pd.get_dummies(df_collisions, prefix=['addrtype', 'collisiontype', 'underinfl', 'hitparkedcar'], prefix_sep='_', columns=object_columns, drop_first=True)

##### Label encoding

In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
#WEATHER
df_collisions.WEATHER.value_counts()

Clear                       116216
Raining                      33145
Overcast                     27714
Unknown                      15091
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [22]:
#ROADCOND
df_collisions.ROADCOND.value_counts()

Dry               129522
Wet                47474
Unknown            15078
Ice                 1209
Snow/Slush          1004
Other                132
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64

In [23]:
#COLLISIONTYPE
df_collisions.LIGHTCOND.value_counts()

Daylight                    121307
Dark - Street Lights On      48507
Unknown                      13473
Dusk                          5902
Dawn                          2502
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Other                          235
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

Let's label enconde this 3 columns:

In [24]:
le_weather = LabelEncoder()
df_collisions['WEATHER'] = le_weather.fit_transform(df_collisions['WEATHER'])

le_roadcond = LabelEncoder()
df_collisions['ROADCOND'] = le_roadcond.fit_transform(df_collisions['ROADCOND'])

le_lightcond = LabelEncoder()
df_collisions['LIGHTCOND'] = le_lightcond.fit_transform(df_collisions['LIGHTCOND'])

df_collisions[['WEATHER', 'ROADCOND', 'LIGHTCOND']].head(3)

,WEATHER,ROADCOND,LIGHTCOND
0,4,8,5
1,6,8,2
2,4,0,5


In [25]:
print(df_collisions.shape)
print(df_collisions.info())

(194673, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 21 columns):
SEVERITYCODE                194673 non-null int64
PERSONCOUNT                 194673 non-null int64
PEDCOUNT                    194673 non-null int64
PEDCYLCOUNT                 194673 non-null int64
VEHCOUNT                    194673 non-null int64
WEATHER                     194673 non-null int64
ROADCOND                    194673 non-null int64
LIGHTCOND                   194673 non-null int64
addrtype_Block              194673 non-null uint8
addrtype_Intersection       194673 non-null uint8
collisiontype_Cycles        194673 non-null uint8
collisiontype_Head On       194673 non-null uint8
collisiontype_Left Turn     194673 non-null uint8
collisiontype_Other         194673 non-null uint8
collisiontype_Parked Car    194673 non-null uint8
collisiontype_Pedestrian    194673 non-null uint8
collisiontype_Rear Ended    194673 non-null uint8
collisiontype_Right Turn  

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize

In [27]:
# Dataset is imbalanced, so lets grab samples and balance that using undersampling technique
df_collisions.SEVERITYCODE.value_counts()

df_shuffled = df_collisions.sample(frac=1)

df_severitycode_2 = df_shuffled.loc[df_shuffled['SEVERITYCODE'] == 2]
df_severitycode_1 = df_shuffled.loc[df_shuffled['SEVERITYCODE'] == 1].sample(n=58188)

df_collisions = pd.concat([df_severitycode_1, df_severitycode_2])
df_collisions.SEVERITYCODE.value_counts()

2    58188
1    58188
Name: SEVERITYCODE, dtype: int64

In [28]:
X = df_collisions.drop('SEVERITYCODE', axis=1)

X = normalize(X)

y = df_collisions[['SEVERITYCODE']]

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Modeling

In [29]:
# Logistic regression
lr_model = LogisticRegression()
lr_model.fit(x_train, y_train)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [30]:
# Knearest-neighbors (KNN)
scores = np.zeros(21)
for k in range(1, 21):
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(x_train, y_train)
    scores[k-1] = knn_model.score(x_test, y_test)
best_knn_score = scores.max()
best_k = scores.argmax()+1

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/_

### Evaluation

In [31]:
print("Logistic regression score: " + str(lr_model.score(x_test, y_test)))
print("KNN score: " + str(best_knn_score) + " with K=" + str(best_k)) 

Logistic regression score: 0.705697018302638
KNN score: 0.7041503165010168 with K=17
